In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import chardet

file_path = r"D:\pythondata\★_재배치_회수.csv"

# 인코딩 감지
with open(file_path, 'rb') as f:
    rawdata = f.read(10000)
    detected_encoding = chardet.detect(rawdata)['encoding']
    print(f"✅ 감지된 인코딩: {detected_encoding}")

# 파일 처음 몇 줄 미리 보기 (정확한 구분자 파악)
with open(file_path, 'r', encoding=detected_encoding, errors='replace') as f:
    for i in range(5):
        print(f.readline())

✅ 감지된 인코딩: UTF-8-SIG
자전거번호	회수대여소	회수대여소명	회수일시	회수시간

SPB-42889	102	 망원역 1번출구 앞	2023-01-01	4:41:10 PM

SPB-41193	102	 망원역 1번출구 앞	2023-01-01	4:43:04 PM

SPB-80219	102	 망원역 1번출구 앞	2023-01-01	8:21:17 PM

SPB-30519	102	 망원역 1번출구 앞	2023-01-01	8:22:03 PM



In [3]:
import pandas as pd
import os

# 1. CSV 파일 경로 지정
file_path = r"D:\pythondata\★_재배치_회수.csv"

# 2. CSV 파일 안전하게 불러오기
df = pd.read_csv(
    file_path,
    sep="\t",
    encoding="utf-8-sig",
    dtype={"회수대여소": str}  # 혼합 타입 방지
)

# 3. 날짜 + 시간 합치기
df["회수일시"] = pd.to_datetime(df["회수일시"] + " " + df["회수시간"])

# 4. 정렬
df = df.sort_values(by=["회수대여소", "회수일시"]).reset_index(drop=True)

# 5. 방문코드 생성 (30분 이상 차이 날 때 새로운 방문코드)
visit_code = []
visit_index = 1
visit_code.append(f"A{visit_index:04d}")

for i in range(1, len(df)):
    time_diff = df.loc[i, "회수일시"] - df.loc[i - 1, "회수일시"]
    if time_diff.total_seconds() > 1800:
        visit_index += 1
    visit_code.append(f"A{visit_index:04d}")

df["방문코드"] = visit_code

# 6. 순번 부여
df["순번"] = df.groupby("방문코드").cumcount() + 1

# 7. 방문코드별 자전거대수 계산
df["자전거대수"] = df.groupby("방문코드")["회수대여소"].transform("count")

# 8. 요약 테이블 생성
summary = df.groupby("방문코드")["회수대여소"].count().reset_index()
summary.columns = ["방문코드", "레코드수"]

# 9. 📤 결과 저장
# 9-1. 전체데이터는 CSV로 저장 (행 수 제한 없음)
output_csv_path = r"D:\pythondata\회수_방문코드_전체데이터.csv"
df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

# 9-2. 요약 데이터는 Excel 저장
output_summary_path = r"D:\pythondata\회수_방문코드_요약.xlsx"
with pd.ExcelWriter(output_summary_path, engine="openpyxl") as writer:
    summary.to_excel(writer, sheet_name="요약", index=False)

# 완료 메시지
print("✅ CSV 저장 완료:", output_csv_path)
print("✅ 요약 엑셀 저장 완료:", output_summary_path)

✅ CSV 저장 완료: D:\pythondata\회수_방문코드_전체데이터.csv
✅ 요약 엑셀 저장 완료: D:\pythondata\회수_방문코드_요약.xlsx
